In [1]:
import numpy as np
from collections import defaultdict
import random
from numpy import random
import networkx as nx
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%run Data_center_setup.ipynb

import gurobipy as gp
from gurobipy import GRB
from gurobipy import*
from gurobipy import quicksum


## Quantum Part CQM
from dwave.system import DWaveSampler, LeapHybridSampler, LeapHybridCQMSampler
from dwave.system.composites import EmbeddingComposite
import dwave.inspector
import dimod

from dimod.binary.binary_quadratic_model import BinaryQuadraticModel, Binary, Spin, as_bqm
from dimod.discrete.discrete_quadratic_model import DiscreteQuadraticModel
#function set
%run Func_Set.ipynb

[[0. 1. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 1. 0.]]


In [ ]:
round = 0
t_upper = np.inf
t_lower = -np.inf
threshold = 0.5


def master_problem_setup():
    t_bits = 40
    t_decimal = 4


    cqm = dimod.ConstrainedQuadraticModel()
    u_name_iterable = ['dischar','char']

    u = [[dimod.Binary(f'u_{u_name}_{t}') for t in range(T)] for u_name in u_name_iterable]
    u_coefficient = np.zeros((len(u_name_iterable), T))

    x_chiller = [[dimod.Binary(f'x_chiller_#{j}_{t}') for t in range(T)] for j in range(J_chiller)]
    x_chiller_coefficient = np.zeros((J_chiller, T))

    x_tower = [[dimod.Binary(f'x_tower_#{j}_{t}') for t in range(T)] for j in range(J_tower)]
    x_tower_coefficient = np.zeros((J_tower, T))

    t = [dimod.Binary(f't_{i}') for i in range(t_bits)]
    t_coefficient = np.power(2, np.arange(-t_decimal,t_bits - t_decimal), dtype=float)



    #objective function 
    cqm.set_objective(sum(t[i]*t_coefficient[i] for i in range(t_bits)) + \
                    sum(sum(u[i][t] * u_coefficient[i,t] for t in range(T)) for i in range(len(u_name_iterable))) + \
                    sum(sum(x_chiller[j][t] * x_chiller_coefficient[j,t] for t in range(T)) for j in range(J_chiller)) + \
                    sum(sum(x_tower[j][t] * x_tower_coefficient[j,t] for t in range(T)) for j in range(J_tower)))
    
    
    
    for order in range(len(contraint_dict)):
        
        lhs = contraint_dict[order][0]
        
        rhs = contraint_dict[order][2]
    
        cqm.add_constraint( sum(lhs[i] * x[i] for i in range(len(QUBO_obj))) <= rhs, label=f'constraint_{order}')


    # record binary variable position
    binary_desiction_dict = {}
    binary_desiction_val_dict = {}
    problem_A_dict = {}
    index_counter = 0

    for position, binary_name in enumerate(cqm.variables):
        binary_desiction_dict.update({binary_name: position})
        binary_desiction_val_dict.update({binary_name: 0})
        
        if not binary_name.split("_")[0] == "t":
            problem_A_dict.update({binary_name: index_counter})
            index_counter += 1
            
    #setup masterproblem x vector:
    master_x_sol = np.zeros(len(problem_A_dict))
    
    
    
    sampler = LeapHybridCQMSampler()    

    candidate_num = 0
    
    try_num = 0
    
    while candidate_num <= 0:
        
        sampleset = sampler.sample_cqm(cqm, label="SDK Examples - Solar Panel Strategy")

        print("Round {}, try {}, {} feasible solutions of {}.".format(round, try_num, sampleset.record.is_feasible.sum(), len(sampleset))) 
        
        try_num += 1
    
    best = sampleset.filter(lambda row: row.is_feasible).first
    
    t_lower = 0
    
    for key, val in best.sample.items():
        #print(key, val)
        if key.split("_")[0]=="t":
            t_lower += val * t_coefficient[int(key.split("_")[-1])]
        else:
            master_x_sol[problem_A_dict[key]] = val
    
    binary_desiction_val_dict.update({key: val})
        
    
    
    return binary_desiction_dict, binary_desiction_val_dict, problem_A_dict, problem_G_dict, problem_constriant_dict, master_x_sol

In [3]:
def Master_problem(round, t_coefficient, threshold, binary_desiction_dict, binary_desiction_val_dict, problem_A_dict, problem_G_dict, problem_constriant_dict, new_consrtaint_dict):

# record continuous variable position
    continuous_decision_list_T = ["p_dis","p_chr","v_vent","e_in"]
    cont_decision_index_counter = 0
    problem_G_dict = {}

    # arrage the position accordingly to decision var with len t
    for var_name in continuous_decision_list_T:
        for t in range(T):
            problem_G_dict.update({f"{var_name}_{t}":  cont_decision_index_counter})
            cont_decision_index_counter += 1
    ##"T_zone"   "T_sup_zone"
    for zone in range(I_zone):
        ##"T_zone"
        for t in range(T + 1):
            problem_G_dict.update({f"T_zone_{zone}_{t}":  cont_decision_index_counter})
            cont_decision_index_counter += 1
            
        ##  "T_sup_zone"
        for t in range(T):
            problem_G_dict.update({f"T_sup_zone_{zone}_{t}":  cont_decision_index_counter})
            cont_decision_index_counter += 1
            
    ## "E_battery_state"
    for t in range(T + 1):
            problem_G_dict.update({f"E_battery_state_{t}":  cont_decision_index_counter})
            cont_decision_index_counter += 1

        

        
    problem_constriant_dict = {}    
    problem_constraint = 18
    problem_constraint_counter = 0
    for line in range(problem_constraint):
        # constraint 10 ~ 12  has i * t constaint
        if line in [9,10,11,16,17]:
            for i in range(I_zone):
                for t in range(T):
            
                    problem_constriant_dict.update({f"constraint_{line}_zone_{i}_{t}":  problem_constraint_counter })
            
                    problem_constraint_counter += 1
                    
        # contraint 4 initalization
        elif line == 3:
            problem_constriant_dict.update({f"constraint_{line}":  problem_constraint_counter})
            
            problem_constraint_counter += 1
        
        # contraint 16 initalization
        elif line == 15:
            for i in range(I_zone):
                problem_constriant_dict.update({f"constraint_{line}_zone_{i}":  problem_constraint_counter })
            
                problem_constraint_counter += 1
            
        #other constarints
        else:
            for t in range(T):
            
                problem_constriant_dict.update({f"constraint_{line}_{t}":  problem_constraint_counter })
            
                problem_constraint_counter += 1   
        
    #print(binary_desiction_dict)
    #print(problem_G_dict)

        
    ########################################################################
    #   
    #    START construct d
    #
    ########################################################################

    d = np.zeros(len(problem_G_dict))

    for t in range(T):
        d[problem_G_dict[f'e_in_{t}']] = 1

    ########################################################################
    #   
    #    START construct A, G, b
    #
    ########################################################################

    # contrust matrix A
    A = np.zeros((len(problem_constriant_dict ),len(problem_A_dict)))
    print(A.shape)

    for t in range(T):
        # (1st constraint)
        
        for j in range(J_chiller):
            A[problem_constriant_dict[f"constraint_{0}_{t}"] , problem_A_dict[f"x_chiller_#{j}_{t}"]] = -beta_chiller_0[j] - beta_chiller_1[j] * M_chiller_rated[j] 
        for j in range(J_tower):
            A[problem_constriant_dict[f"constraint_{0}_{t}"] , problem_A_dict[f"x_tower_#{j}_{t}"]] = -beta_tower_0[j] - beta_tower_1[j] * M_tower_rated[j] 

        # (7th constraints)
        
        A[problem_constriant_dict[f"constraint_{6}_{t}"] , problem_A_dict[f"u_char_{t}"]] = char_b_upperbound 
        
        # (8th constraints)
        
        A[problem_constriant_dict[f"constraint_{7}_{t}"] , problem_A_dict[f"u_dischar_{t}"]] = dis_b_upperbound 
        
        # (9th constraints)
        
        index_of_dis_and_chr = [problem_A_dict[f"u_char_{t}"] , problem_A_dict[f"u_dischar_{t}"]]
        A[problem_constriant_dict[f"constraint_{8}_{t}"],index_of_dis_and_chr] = -1
        
        # (14th constraints)
        for j in range(J_chiller):
            A[problem_constriant_dict[f"constraint_{13}_{t}"],problem_A_dict[f"x_chiller_#{j}_{t}"]] = M_chiller_rated[j] 
            
        # (15th constraints)
        for j in range(J_tower):
            A[problem_constriant_dict[f"constraint_{14}_{t}"],problem_A_dict[f"x_tower_#{j}_{t}"]] = M_tower_rated[j] 
            
    # contrust matrix G
    G = np.zeros((len(problem_constriant_dict ),len(problem_G_dict)))

    # initial setup
    # (4th constraint)
    G[problem_constriant_dict[f"constraint_{3}"] , problem_G_dict["E_battery_state_0"]] = Battery_capacity_init

    # 16th constraint)
    for zone in range(I_zone):
        G[problem_constriant_dict[f"constraint_{15}_zone_{zone}"] , problem_G_dict[f"T_zone_{zone}_0"]] = 1 
        
    for t in range(T):
        # (1st constraint)
        G[problem_constriant_dict[f"constraint_{0}_{t}"],problem_G_dict[f"p_dis_{t}"]] = 1/eta_dischar_b
        G[problem_constriant_dict[f"constraint_{0}_{t}"],problem_G_dict[f"p_chr_{t}"]] = -eta_char_b
        for zone in range(I_zone):
            G[problem_constriant_dict[f"constraint_{0}_{t}"] , problem_G_dict[f"T_zone_{zone}_{t}"]] = - beta_pump_1 * Chi[zone] * (v_sup[t] - v_out[t]) * C_air_normal / ((T_chiller_water_return[t]  - T_chiller_water_supply[t]) * C_water)
            G[problem_constriant_dict[f"constraint_{0}_{t}"] , problem_G_dict[f"T_sup_zone_{zone}_{t}"]] = beta_pump_1 * Chi[zone] * v_sup[t] * C_air_normal / ((T_chiller_water_return[t]  - T_chiller_water_supply[t]) * C_water)
            G[problem_constriant_dict[f"constraint_{0}_{t}"] , problem_G_dict[f"v_vent_{t}"]] = - beta_vent_1
            
        # (3rd constraint)
        G[problem_constriant_dict[f"constraint_{2}_{t}"] ,problem_G_dict[f"p_dis_{t}"]] = 1/eta_dischar_b
        G[problem_constriant_dict[f"constraint_{2}_{t}"] ,problem_G_dict[f"p_chr_{t}"]] = -eta_char_b
        G[problem_constriant_dict[f"constraint_{2}_{t}"],problem_G_dict[f"E_battery_state_{t}"]] = -1
        G[problem_constriant_dict[f"constraint_{2}_{t}"],problem_G_dict[f"E_battery_state_{t + 1}"]] = 1
        
        # (10th constraint)
        for zone in range(I_zone):
            G[problem_constriant_dict[f"constraint_{9}_zone_{zone}_{t}"] , problem_G_dict[f"T_zone_{zone}_{t}"]] = - 1 + np.sum(adjacent_zone_map[zone , :]) /(C_air_normal * R_air_ij) + m_air_in[zone] * c_s_air[zone] / C_air_normal
            
            G[problem_constriant_dict[f"constraint_{9}_zone_{zone}_{t}"] , problem_G_dict[f"T_zone_{zone}_{t + 1}"]] = 1
            
            G[problem_constriant_dict[f"constraint_{9}_zone_{zone}_{t}"] , problem_G_dict[f"T_zone_{zone}_{t + 1}"]] = - m_air_in[zone] * c_s_air[zone] / C_air_normal
            
            adjacent_zone = np.where(adjacent_zone_map[zone ,:])[0]
            for adj_zone in adjacent_zone:
                G[problem_constriant_dict[f"constraint_{9}_zone_{adj_zone}_{t}"] , problem_G_dict[f"T_zone_{adj_zone}_{t}"]] = - 1 / (C_air_normal * R_air_ij)
                
        # (2nd constraint)
        G[problem_constriant_dict[f"constraint_{1}_{t}"] , problem_G_dict[f"e_in_{t}"]] = 1
        
        # (5th constraint)
        G[problem_constriant_dict[f"constraint_{4}_{t}"] , problem_G_dict[f"E_battery_state_{t + 1}"]] = -1
        
        # (6th constraint)
        G[problem_constriant_dict[f"constraint_{5}_{t}"] , problem_G_dict[f"E_battery_state_{t + 1}"]] = 1
        
        # (7th constraint)
        G[problem_constriant_dict[f"constraint_{6}_{t}"] , problem_G_dict[f"p_chr_{t}"]] = 1
        
        # (8th constraint)
        G[problem_constriant_dict[f"constraint_{7}_{t}"] , problem_G_dict[f"p_dis_{t}"]] = 1
        
        
        for zone in range(I_zone):
            
            # (11th constraint)
            G[problem_constriant_dict[f"constraint_{10}_zone_{zone}_{t}"] , problem_G_dict[f"T_zone_{zone}_{t + 1}"]] = - 1
        
            # (12th constraint)
            G[problem_constriant_dict[f"constraint_{11}_zone_{zone}_{t}"] , problem_G_dict[f"T_zone_{zone}_{t + 1}"]] = 1
            
            # (11a "17" th constraint)
            G[problem_constriant_dict[f"constraint_{16}_zone_{zone}_{t}"] , problem_G_dict[f"T_sup_zone_{zone}_{t}"]] = -1
            
            # (12a "18" th constraint)
            G[problem_constriant_dict[f"constraint_{17}_zone_{zone}_{t}"] , problem_G_dict[f"T_sup_zone_{zone}_{t}"]] = 1
            
        # (13th constraint)
        G[problem_constriant_dict[f"constraint_{12}_{t}"] , problem_G_dict[f"v_vent_{t}"]] = 1
        
        
        
        for zone in range(I_zone):
            
            # (14th constraint)
            G[problem_constriant_dict[f"constraint_{13}_{t}"] , problem_G_dict[f"T_zone_{zone}_{t}"]] = - Chi[zone] * (v_sup[t] - v_out[t]) * C_air_normal / ((T_chiller_water_return[t]  - T_chiller_water_supply[t]) * C_water)
            G[problem_constriant_dict[f"constraint_{13}_{t}"] , problem_G_dict[f"T_sup_zone_{zone}_{t}"]] = Chi[zone] * v_sup[t] * C_air_normal / ((T_chiller_water_return[t]  - T_chiller_water_supply[t]) * C_water)
            
            # (15th constraint)
            G[problem_constriant_dict[f"constraint_{14}_{t}"] , problem_G_dict[f"T_zone_{zone}_{t}"]] = - Chi[zone] * (v_sup[t] - v_out[t]) * C_air_normal / ((T_tower_water_return[t]  - T_tower_water_supply[t]) * C_water)
            G[problem_constriant_dict[f"constraint_{14}_{t}"] , problem_G_dict[f"T_sup_zone_{zone}_{t}"]] = Chi[zone] * v_sup[t] * C_air_normal / ((T_tower_water_return[t]  - T_tower_water_supply[t]) * C_water)

        
    # contrust vector b
    b = np.zeros((len(problem_constriant_dict ),1))

    # (4rd constraint)
    b[problem_constriant_dict[f"constraint_{3}"],0] = Battery_capacity_init

    # (16th constraint)
    for zone in range(I_zone):
        b[problem_constriant_dict[f"constraint_{15}_zone_{zone}"] , 0] = T_zone_initial[zone//I_zone_length , zone%I_zone_length]

    for t in range(T):
        # (1st constraint)
        b[problem_constriant_dict[f"constraint_{0}_{t}"],0] = e_sup[t] + beta_vent_0  - beta_vent_1 * v_vent_lowerbound[t] + \
        beta_pump_0 + beta_pump_1 * T_out[t] * v_sup[t]  * C_air_normal/ ((T_tower_water_return[t]  - T_tower_water_supply[t]) * C_water) + e_server[t] + e_solar[t]
        
        # (3rd constraint)
        b[problem_constriant_dict[f"constraint_{2}_{t}"],0] = 0
        
        # (10th constraint)
        for zone in range(I_zone):
            b[problem_constriant_dict[f"constraint_{9}_zone_{zone}_{t}"] , 0] = theta_heat[zone,t]/c_s_air[zone]
                
        # (2nd constraint)
        b[problem_constriant_dict[f"constraint_{1}_{t}"] , 0] = 0
        
        # (5th constraint)
        b[problem_constriant_dict[f"constraint_{4}_{t}"] , 0] = -Battery_capacity_upbound 
        
        # (6th constraint)
        b[problem_constriant_dict[f"constraint_{5}_{t}"] , 0] = Battery_capacity_lowbound
        
        # (7th constraint)
        b[problem_constriant_dict[f"constraint_{6}_{t}"] , 0] = 0
        
        # (8th constraint)
        b[problem_constriant_dict[f"constraint_{7}_{t}"] , 0] = 0
        
        # (9th constraint)
        b[problem_constriant_dict[f"constraint_{8}_{t}"] , 0] = -1
        
        
        for zone in range(I_zone):
            
            # (11th constraint)
            b[problem_constriant_dict[f"constraint_{10}_zone_{zone}_{t}"] , 0] = -T_datacenter_upperbound[zone , t]
        
            # (12th constraint)
            b[problem_constriant_dict[f"constraint_{11}_zone_{zone}_{t}"] , 0] = T_datacenter_lowerbound[zone , t]
            
            # (11a "17" th constraint)
            b[problem_constriant_dict[f"constraint_{16}_zone_{zone}_{t}"] , 0] = - T_AC_upperbound[zone , t]
            
            # (12a "18" th constraint)
            b[problem_constriant_dict[f"constraint_{17}_zone_{zone}_{t}"] , 0] = T_AC_lowerbound[zone , t]
            
        # (13th constraint)
        b[problem_constriant_dict[f"constraint_{12}_{t}"] , 0] = v_vent_lowerbound[t]
        
        
        for zone in range(I_zone):
            
            # (14th constraint)
            b[problem_constriant_dict[f"constraint_{13}_{t}"] , 0] = - T_out[t] * v_sup[t]* C_air_normal / ((T_chiller_water_return[t]  - T_chiller_water_supply[t]) * C_water)
            
            # (15th constraint)
            b[problem_constriant_dict[f"constraint_{14}_{t}"] , 0] = - T_out[t] * v_sup[t] * C_air_normal / ((T_tower_water_return[t]  - T_tower_water_supply[t]) * C_water)
                 
    
    return 0

IndentationError: expected an indented block (<ipython-input-3-5c9c23420c00>, line 224)